In [1]:
import pickle
from misc import *
import SYCLOP_env as syc
from RL_brain_b import DeepQNetwork


In [2]:
import cv2
import time

In [3]:
from mnist import MNIST

mnist = MNIST('/home/bnapp/datasets/mnist/')
images, labels = mnist.load_training()
# some_mnistSM =[ cv2.resize(1.+np.reshape(uu,[28,28]), dsize=(256, 256)) for uu in images[:2]]#[:4096]]   
some_samples_for_setup= prep_mnist_padded_images(2)
           


In [4]:
# run_dir = 'saved_runs/run_syclop_generic1.py_noname_1576060868_0/' #padded mnist beta 0.1 speed penalty 5
# result_type = 'nwk2.nwk'

# run_dir = 'saved_runs/run_syclop_generic1.py_noname_1576147784_0/' #padded mnist beta 0.1 speed penalty 20


run_dir = 'saved_runs/run_syclop_generic1.py_noname_1576403573_0/'  #padded mnist beta 0.1 speed penalty 0
result_type = 'tempX_1.nwk'


In [5]:
hp = HP()
hp.mem_depth=1
hp.logmode=False
batch_size=256
action_space_size=9
# images = some_mnistSM
number_of_images = len(images)
reward = syc.Rewards()
observation_size = 256*4
RL = DeepQNetwork(action_space_size, observation_size*hp.mem_depth,#sensor.frame_size+2,
                  reward_decay=0.99,
                  e_greedy=1-1e-9,
                  e_greedy0=1-1e-9,
                  replace_target_iter=10,
                  memory_size=100000,
                  e_greedy_increment=0.0001,
                  learning_rate=0.0025,
                  double_q=False,
                  dqn_mode=True,
                  state_table=np.zeros([1,observation_size*hp.mem_depth]),
                  soft_q_type='boltzmann',
                  beta=0.1
                  )




debug n_features: 1024
debug: (?, 1024)
debug: (?, 400)
debug: (?, 200)
debug: (?, 200)
debug: (?, 200)
debug n_features: 1024
debug: (?, 1024)
debug: (?, 400)
debug: (?, 200)
debug: (?, 200)
debug: (?, 200)


In [6]:

def local_observer(sensor,agent):
    if hp.logmode:
        normfactor=1.0
    else:
        normfactor = 1.0/256.0
    return normfactor*np.concatenate([relu_up_and_down(sensor.central_dvs_view),
            relu_up_and_down(cv2.resize(1.0*sensor.dvs_view, dsize=(16, 16), interpolation=cv2.INTER_AREA))])

observation = np.random.uniform(0,1,size=[hp.mem_depth, observation_size])


In [7]:
scene_bb = [None]*batch_size
sensor_bb =[None]*batch_size
agent_bb = [None]*batch_size
action_bb = [None]*batch_size
action_list_bb =  [None]*batch_size
q_list_bb =  [None]*batch_size
observation_bb = [None]*batch_size

with open(run_dir+'/hp.pkl','rb') as f:
                this_hp = pickle.load(f)

for bb in range(batch_size):
    scene_bb[bb] = syc.Scene(frame_list=some_samples_for_setup[0:1])
    sensor_bb[bb] = syc.Sensor()
    agent_bb[bb] = syc.Agent(max_q = [scene_bb[bb].maxx-sensor_bb[bb].hp.winx,scene_bb[bb].maxy-sensor_bb[bb].hp.winy])
    agent_bb[bb].hp.action_space = this_hp.agent.action_space


In [8]:

RL.dqn.load_nwk_param(run_dir+'/'+ result_type) 
with open(run_dir+'/hp.pkl','rb') as f:
          this_hp = pickle.load(f)
          hp.fading_mem = this_hp.fading_mem +0.0 #to avoid assignment by address
size=(28,28)
offset=(0,0)
action_records=[]
q_records=[]
observation_feeder=np.zeros([batch_size,1024])
for image_num,image in enumerate(images):
    step = 0
    episode = 0
for batch_num in range(len(images)//batch_size):
    for bb in range(batch_size):
        action_list_bb[bb] = []
#         q_list_bb[bb] = []
        observation_bb[bb] = np.random.uniform(0,1,size=[hp.mem_depth, observation_size])
        observation_bb[bb] = np.random.uniform(0,1,size=[hp.mem_depth, observation_size])
#         scene_bb[bb].current_frame = image_num[bb]
    
    #### sizing story:
        image_resized=cv2.resize(0.0+np.reshape(images[batch_num*batch_size+bb],[28,28]), dsize=size)
        scene_bb[bb].image = build_mnist_padded([image_resized],y_size=size[1],x_size=size[0],offset=offset)
#         scene_bb[bb].image = build_mnist_padded([images[batch_num*batch_size+bb]])

        agent_bb[bb].reset()
        agent_bb[bb].q_ana[1]=128./2.-32
        agent_bb[bb].q_ana[0]=128./2-32
        agent_bb[bb].q = np.int32(np.floor(agent_bb[bb].q_ana))

        sensor_bb[bb].reset()
        sensor_bb[bb].update(scene_bb[bb], agent_bb[bb])
        sensor_bb[bb].update(scene_bb[bb], agent_bb[bb])
        
    time1=time.time()
    for step_prime in range(1000):
        deep_time1=time.time()
#         action = RL.choose_action(observation.reshape([-1]))
        for bb in range(batch_size):
            observation_feeder[bb,:]=observation_bb[bb].reshape([1,-1])
        oo = RL.dqn.eval_eval(observation_feeder)        
        boltzmann_measure = np.exp(RL.beta * (oo-np.max(oo,axis=1).reshape([-1,1]))) #todo here substracted max to avoid exponent exploding. need to be taken into a separate function!
        boltzmann_measure = boltzmann_measure / np.sum(boltzmann_measure, axis=1).reshape([-1,1])
        for bb in range(batch_size):
            action_bb[bb] = np.random.choice(list(range(RL.n_actions)),1, p=boltzmann_measure[bb,:].reshape([-1]))[0]
        
#         action_bb= [a for a in np.argmax(oo,axis=1)]
        deep_time2=time.time()
        shallow_time1=time.time()
        
        for bb in range(batch_size):
            agent_bb[bb].act(action_bb[bb])
            action_list_bb[bb].append(action_bb[bb])
#             q_list_bb[bb].append(agent_bb[bb].q_ana)
            sensor_bb[bb].update(scene_bb[bb],agent_bb[bb])
            observation_bb[bb] *= hp.fading_mem
            observation_bb[bb] += local_observer(sensor_bb[bb], agent_bb[bb])  # todo: generalize
        shallow_time2=time.time()
#         print('deep:',deep_time2-deep_time1,'shallow:',shallow_time2-shallow_time1)
    time2=time.time()
    
    print('batch num:',batch_num,'wall time consumed:',time2-time1)
    for bb in range(batch_size):
        action_records.append(action_list_bb[bb])
#         q_records.append(q_list_bb[bb])


batch num: 0 wall time consumed: 39.24245738983154
batch num: 1 wall time consumed: 38.755786180496216
batch num: 2 wall time consumed: 38.64829134941101
batch num: 3 wall time consumed: 38.89785671234131
batch num: 4 wall time consumed: 38.728580713272095
batch num: 5 wall time consumed: 38.78235936164856
batch num: 6 wall time consumed: 38.581660747528076
batch num: 7 wall time consumed: 39.75066304206848
batch num: 8 wall time consumed: 39.06776213645935
batch num: 9 wall time consumed: 38.79161477088928
batch num: 10 wall time consumed: 38.849507093429565
batch num: 11 wall time consumed: 39.05150532722473
batch num: 12 wall time consumed: 39.012853384017944
batch num: 13 wall time consumed: 38.74488806724548
batch num: 14 wall time consumed: 38.79856014251709
batch num: 15 wall time consumed: 38.77446150779724
batch num: 16 wall time consumed: 38.767223834991455
batch num: 17 wall time consumed: 39.92315196990967
batch num: 18 wall time consumed: 38.765279054641724
batch num: 19 w

batch num: 157 wall time consumed: 39.351991415023804
batch num: 158 wall time consumed: 39.3531379699707
batch num: 159 wall time consumed: 39.141111612319946
batch num: 160 wall time consumed: 39.34543538093567
batch num: 161 wall time consumed: 38.92388987541199
batch num: 162 wall time consumed: 39.067177295684814
batch num: 163 wall time consumed: 39.304216623306274
batch num: 164 wall time consumed: 39.302462100982666
batch num: 165 wall time consumed: 39.244834423065186
batch num: 166 wall time consumed: 38.98465609550476
batch num: 167 wall time consumed: 39.18151664733887
batch num: 168 wall time consumed: 39.60953235626221
batch num: 169 wall time consumed: 39.37199854850769
batch num: 170 wall time consumed: 39.077603340148926
batch num: 171 wall time consumed: 39.33176231384277
batch num: 172 wall time consumed: 40.10268592834473
batch num: 173 wall time consumed: 40.035783529281616
batch num: 174 wall time consumed: 39.42797064781189
batch num: 175 wall time consumed: 39.0

In [9]:
len(action_records)

59904

In [10]:
with open('mnist_padded_b0p1_v0_X28_Tx0y0_act_full1.pkl','wb') as f:
    pickle.dump([action_records[:30000],labels[:30000]],f)
    
with open('mnist_padded_b0p1_v0_X28_Tx0y0_act_full2.pkl','wb') as f:
    pickle.dump([action_records[30000:],labels[30000:]],f)

In [11]:
np.shape(sensor_bb[0].frame_view)

(64, 64)

In [12]:
agent_bb[0].q_ana

array([20., 27.])